In [1]:
import numpy as np 
from collections import Counter


# Cross entropy loss :
$$L = \sum_{c} p_c * log_2(p_c)$$

In [13]:
#implement Cross_entropy methos, globally, we could also use the Gini, instead of cross entropy as loss function
def entropy_method(y):
    class_count = np.bincount(y) #calcluate the number of occurency of all class label
    proportions = class_count/len(y)
    return - np.sum([p*np.log2(p) for p in proportions if p > 0])


# Node class
Now define a class that will store the information for our node, if we are in the middle , so the node is not a leaf, we want to store the best feature to split and the treshold to use.
We want to store alse the left and right child node, becase we need the to calculate the loss of the split, if we are at a leaf node we store the most common class labels that will be the value of that leaf.


In [14]:

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
  #we just store the infromation      
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value 
#now we define a function that tell us if our node is a leaf node 
    def is_leaf_node(self):
        return self.value is not None
#if the node is a leaf, it will return a value so self.value will not be None, so this function will return True, meaning that the node is a leaf



# Decision Tree classifier

In [15]:
class DecisionTree: 
    #in practice now we do a greedy search over oll the features but we can also loop only in a subset of feature in random way --> random forest
    def __init__(self, min_sample_split=2, max_depth=100, n_feats=None):
        self.min_sample_split = min_sample_split #if there are too few element in a node stop split
        self.max_depth = max_depth
        self.n_feats = n_feats
        self.root = None #the starting node

# Now we define all the helper funtion that our fit and predict methonds need


    def _most_common_label(self, y):
        counter = Counter(y) # similar to the np.bincount
        most_common = counter.most_common(1)[0][0]
      #very most common tuple in the list, and we want the first elment of that tuple, in the touple there is the value store and also the number of occurencies, and we only want the value
        return most_common




#split the tree and save the childres
    def _split(self, X_column, split_threh):
        left_idx = np.argwhere(X_column <= split_threh).flatten()
        right_idx = np.argwhere(X_column > split_threh).flatten()
        return left_idx, right_idx




# infromation gain to use the search for the best split
    def _information_gain(self, y, X_column, split_threh):
        #parent entropy
        parent_en = entropy_method(y)
        #generate split
        left_idx, right_idx = self._split(X_column, split_threh)
        
        if len(left_idx)==0 or len(right_idx)==0 : 
            # we don't gain information so we return 0
            return 0
        
        #weighted average of the cross entropy of the childs
        n = len(y)
        n_l = len(left_idx)
        n_r = len(right_idx)
        e_l,e_r = entropy_method(y[left_idx]), entropy_method(y[right_idx])
        child_en = (n_l/n)*e_l + (n_r/n)*e_r

        #return information gain
        info_gain = parent_en - child_en
        return info_gain




# the function for our greedy search
    def _best_criteria(self, X, y, feat_idxs): 
        #go aver all the feaures and all the features' values and calculate the information gain
        best_gain = -1
        split_idx, split_threh = None, None
        for feat_idx in feat_idxs:
            X_column = X[:,feat_idx] #look only at the column of the current feature
            thresholds = np.unique(X_column) #go over all possible threshold but don't check the same value twice
            for threshold in thresholds :
                gain = self._information_gain(y, X_column, threshold)
                #we want to maximize the gain
                if gain > best_gain: 
                    best_gain = gain
                    split_idx = feat_idx
                    split_threh = threshold
        #return the best split index(the feature to split) and
        # the best split threshold(who to split the feature cloumn) in order
        #to obain the greater gain in information
        return split_idx, split_threh  




    #our main helper funtion
    def _grow_tree(self, X, y, depth=0):
    # depth is 0 in the beginning but we need to keep track of that

        n_sample, n_feature = X.shape
        n_labels = len(np.unique(y))
        
    # stopping criteria 
        #if one of the followinf is true we return the leaf node with the value of the most common label
        if (depth >= self.max_depth or n_labels == 1  or n_sample < self.min_sample_split): 
                                      
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        
     #if the node did't meet the stopping criterias:
        feat_idxs = np.random.choice(n_feature, self.n_feats, replace=False)
        #(the range in wich made the choice, the length of the vector of choice=number of choice to make, we don't want to have the same index multiple time )

        #greedy search--> split to obtain the best gain in infromation
        best_feat, best_thresh = self._best_criteria(X,y,feat_idxs)
        left_idx, right_idx = self._split(X[:, best_feat], best_thresh) 
        
        #split inside the best column by recursively recoll grow_tree inside the cloumn split(the new node)
        left = self._grow_tree(X[left_idx,:], y[left_idx], depth+1)
        right = self._grow_tree(X[right_idx,:], y[right_idx], depth+1)
        return Node(best_feat, best_thresh, left, right) 




# helper funtion for the predict method
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)



# Define fit and predict methods
    def fit(self,X,y):
        #Now if we don't specify how many feture we want, we use all the featueres in X 
        # that correspond to the second dimension X[1] bc X = n x d where d are the number of all the features
        #is we specify the n_feats we take the minimum btw n_feats and the number of all the features in X
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
        #grow tree
        self.root = self._grow_tree(X,y)

    def predict(self, X):
        #the root is where we start
        return np.array([self._traverse_tree(x,self.root) for x in X] )
       


# Test our Decision Tree

In [17]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split


def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

data = datasets.load_breast_cancer()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)
    
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)

print ("Accuracy:", acc)

Accuracy: 0.9210526315789473
